In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install fvcore tqdm matplotlib

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61396 sha256=92c80c5231982685533f26a98107e1303cde8976075db4664809e04bddd720d6
  Stored in directory: /root/.cache/pip/wheels/65/71/95/3b8fde5c65c6e4a806e0867c1651dcc71a1cb2f3430e8f355f
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31528 sha256=e5d1a6efd92b4720af9dca85050b5565abed0a0e65da41417a549d20eb196dfb
  Stored in directory: /root/.cache/pip/wheels/ba/5e/16/6117f8fe7e9c0c161a795e10d94645ebcf301ccbd01f66d8ec
Successfully built fvcore iopath


In [ ]:
!git clone https://github.com/xpuria/MLDL2024_project.git

Cloning into 'MLDL2024_project'...
remote: Enumerating objects: 141, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 141 (delta 45), reused 50 (delta 22), pack-reused 57 (from 1)
Receiving objects: 100% (141/141), 45.94 KiB | 1.84 MiB/s, done.
Resolving deltas: 100% (61/61), done.


In [ ]:
# Create project directories
!mkdir -p datasets/cityscapes datasets/gta5 checkpoints plots

# First, let's extract the datasets
import os
import zipfile
from pathlib import Path

# Extract Cityscapes
print("Extracting Cityscapes...")
cityscapes_zip = '/content/drive/MyDrive/Cityscapes.zip'
cityscapes_dir = '/content/datasets/cityscapes'

with zipfile.ZipFile(cityscapes_zip, 'r') as zip_ref:
    zip_ref.extractall(cityscapes_dir)

# Extract GTA5
print("Extracting GTA5...")
gta5_zip = '/content/drive/MyDrive/GTA5.zip'
gta5_dir = '/content/datasets/gta5'

with zipfile.ZipFile(gta5_zip, 'r') as zip_ref:
    zip_ref.extractall(gta5_dir)

# Verify the structure
def print_directory_structure(path, level=0):
    indent = '  ' * level
    print(f"{indent}{os.path.basename(path)}/")
    if level < 2:  # Limit depth to avoid too much output
        for item in os.listdir(path):
            item_path = os.path.join(path, item)
            if os.path.isdir(item_path):
                print_directory_structure(item_path, level + 1)

print("\nDataset structure:")
print_directory_structure('/content/datasets')

# Print dataset statistics
print("\nDataset statistics:")
print("Cityscapes:")
print(f"Total files: {len(list(Path(cityscapes_dir).rglob('*.*')))}")
print("\nGTA5:")
print(f"Total files: {len(list(Path(gta5_dir).rglob('*.*')))}")

Extracting Cityscapes...
Extracting GTA5...

Dataset structure:
datasets/
  gta5/
    GTA5/
  cityscapes/
    Cityscapes/

Dataset statistics:
Cityscapes:
Total files: 6216

GTA5:
Total files: 5000


In [ ]:
!pip uninstall -y sympy
!pip install sympy==1.12

Found existing installation: sympy 1.13.1
Uninstalling sympy-1.13.1:
  Successfully uninstalled sympy-1.13.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 43.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu121 requires sympy==1.13.1; python_version >= "3.9", but you have sympy 1.12 which is incompatible.


In [ ]:
import torch
from torch.utils.data import DataLoader
import numpy as np
from MLDL2024_project.datasets.cityscapes import CityscapesDataset
from MLDL2024_project.datasets.gta5 import GTA5Dataset
from MLDL2024_project.models.bisenet.build_bisenet import BiSeNet
from MLDL2024_project.train_adv import train_adversarial
from MLDL2024_project.train_adv import CustomDiscriminator
from torchvision import transforms
from torch import nn

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cuda


In [ ]:
augment_1 = transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.3)
GTA5_train_dataset = GTA5Dataset(
        root_dir='/content/datasets/gta5/GTA5',
        height=720,
        width=1280,
        augment = augment_1
    )
GTA5_train_dataloader_augmented_1 = DataLoader(GTA5_train_dataset, batch_size=2, shuffle=True, num_workers=4)

cityscapes_test_dataset = CityscapesDataset(
        root_path='/content/datasets/cityscapes/Cityscapes/Cityspaces',
        split='val',
        img_size=(512, 1024)
    )
cityscapes_test_dataloader = DataLoader(cityscapes_test_dataset, batch_size=2, shuffle=True, num_workers=4)


Cityscapes val Dataset:
Number of samples: 500
Image size: (512, 1024)
Augmentations: False
Cities (3): frankfurt, lindau, munster


In [ ]:
interp_s = nn.Upsample(size=(720, 1280), mode='bilinear')
interp_t = nn.Upsample(size=(512, 1024), mode='bilinear')

In [ ]:
generator = BiSeNet(num_classes=19, context_path='resnet18').to(device)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 165MB/s]
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:00<00:00, 206MB/s]


In [ ]:
gen_criterion = torch.nn.CrossEntropyLoss(ignore_index=255)
gen_optimizer = torch.optim.SGD(generator.parameters(), lr=2.5e-2, momentum=0.9, weight_decay=1e-4)

In [ ]:
discriminator = CustomDiscriminator(num_classes=19).to(device)

In [ ]:
disc_criterion = torch.nn.BCEWithLogitsLoss()
disc_optimizer = torch.optim.Adam(discriminator.parameters(), lr=1e-4, betas=(0.9, 0.99))

In [ ]:
train_mious, val_mious, best_epoch = train_adversarial(
    generator=generator,
    discriminator=discriminator,
    gen_criterion=gen_criterion,
    disc_criterion=disc_criterion,
    gen_optimizer=gen_optimizer,
    disc_optimizer=disc_optimizer,
    source_loader=GTA5_train_dataloader_augmented_1,
    target_loader=cityscapes_test_dataloader,
    source_interp=interp_s ,
    target_interp=interp_t,
    num_classes=19,
    device=device,
    epochs=50,
    source_flag=0,
    target_flag=1,
    lambda_adv=0.001
)

Epoch 1/50 (Val): 100%|██████████| 250/250 [00:16<00:00, 15.48it/s, Loss=1.1946]



Epoch 1/50:
Train Loss: 0.6348, mIoU: 20.67%

Train IoU per class:
road                : 85.15%
sidewalk            : 37.94%
building            : 50.54%
wall                : 13.10%
fence               : 0.02%
pole                : 8.05%
traffic light       : 0.00%
traffic sign        : 0.01%
vegetation          : 51.33%
terrain             : 20.12%
sky                 : 83.36%
person              : 0.00%
rider               : 0.00%
car                 : 40.45%
truck               : 2.11%
bus                 : 0.58%
train               : 0.04%
motorcycle          : 0.00%
bicycle             : 0.00%

Val Loss: 1.2363, mIoU: 15.91%

Val IoU per class:
road                : 81.23%
sidewalk            : 8.46%
building            : 55.29%
wall                : 4.11%
fence               : 0.01%
pole                : 0.80%
traffic light       : 0.00%
traffic sign        : 0.00%
vegetation          : 51.47%
terrain             : 9.13%
sky                 : 47.91%
person              : 0.00%


Epoch 2/50 (Val): 100%|██████████| 250/250 [00:16<00:00, 15.46it/s, Loss=0.8992]



Epoch 2/50:
Train Loss: 0.4825, mIoU: 26.15%

Train IoU per class:
road                : 87.83%
sidewalk            : 48.83%
building            : 59.02%
wall                : 26.24%
fence               : 0.96%
pole                : 17.39%
traffic light       : 0.00%
traffic sign        : 0.00%
vegetation          : 60.65%
terrain             : 35.77%
sky                 : 88.45%
person              : 0.00%
rider               : 0.00%
car                 : 55.11%
truck               : 14.96%
bus                 : 0.42%
train               : 1.14%
motorcycle          : 0.00%
bicycle             : 0.00%

Val Loss: 0.8489, mIoU: 22.08%

Val IoU per class:
road                : 84.70%
sidewalk            : 23.00%
building            : 70.54%
wall                : 12.30%
fence               : 1.28%
pole                : 11.71%
traffic light       : 0.00%
traffic sign        : 0.00%
vegetation          : 65.18%
terrain             : 10.79%
sky                 : 70.64%
person              : 

Epoch 3/50 (Val): 100%|██████████| 250/250 [00:16<00:00, 15.57it/s, Loss=1.6117]


...


Epoch 48/50:
Train Loss: 0.1290, mIoU: 74.23%

Train IoU per class:
road                : 96.05%
sidewalk            : 84.75%
building            : 86.68%
wall                : 78.84%
fence               : 64.55%
pole                : 51.33%
traffic light       : 49.88%
traffic sign        : 59.53%
vegetation          : 81.02%
terrain             : 77.86%
sky                 : 95.06%
person              : 50.86%
rider               : 56.04%
car                 : 90.94%
truck               : 90.53%
bus                 : 93.93%
train               : 91.24%
motorcycle          : 69.45%
bicycle             : 41.91%

Val Loss: 0.8937, mIoU: 28.81%

Val IoU per class:
road                : 82.42%
sidewalk            : 20.19%
building            : 76.81%
wall                : 15.94%
fence               : 9.07%
pole                : 21.00%
traffic light       : 14.92%
traffic sign        : 5.25%
vegetation          : 74.23%
terrain             : 14.95%
sky                 : 81.44%
person     


Epoch 49/50 (Val): 100%|██████████| 250/250 [00:16<00:00, 15.49it/s, Loss=1.1590]


Epoch 49/50:
Train Loss: 0.1289, mIoU: 74.22%

Train IoU per class:
road                : 96.06%
sidewalk            : 84.82%
building            : 86.64%
wall                : 78.91%
fence               : 64.46%
pole                : 51.32%
traffic light       : 50.10%
traffic sign        : 59.54%
vegetation          : 80.99%
terrain             : 77.86%
sky                 : 95.04%
person              : 50.69%
rider               : 56.52%
car                 : 90.96%
truck               : 90.53%
bus                 : 94.11%
train               : 91.65%
motorcycle          : 70.25%
bicycle             : 39.76%

Val Loss: 0.8890, mIoU: 28.78%

Val IoU per class:
road                : 81.95%
sidewalk            : 21.39%
building            : 76.65%
wall                : 16.11%
fence               : 9.18%
pole                : 21.85%
traffic light       : 14.79%
traffic sign        : 4.88%
vegetation          : 73.82%
terrain             : 14.74%
sky                 : 80.51%
person     


Epoch 50/50 (Val): 100%|██████████| 250/250 [00:16<00:00, 15.45it/s, Loss=0.8998]



Epoch 50/50:
Train Loss: 0.1285, mIoU: 74.28%

Train IoU per class:
road                : 96.07%
sidewalk            : 84.82%
building            : 86.74%
wall                : 79.04%
fence               : 64.76%
pole                : 51.41%
traffic light       : 50.32%
traffic sign        : 59.63%
vegetation          : 81.07%
terrain             : 77.92%
sky                 : 95.06%
person              : 50.88%
rider               : 56.90%
car                 : 90.96%
truck               : 90.63%
bus                 : 93.91%
train               : 91.73%
motorcycle          : 69.67%
bicycle             : 39.69%

Val Loss: 0.8582, mIoU: 29.28%

Val IoU per class:
road                : 82.37%
sidewalk            : 22.00%
building            : 76.89%
wall                : 16.87%
fence               : 10.30%
pole                : 22.52%
traffic light       : 14.82%
traffic sign        : 6.50%
vegetation          : 74.91%
terrain             : 15.39%
sky                 : 80.29%
person    